In [1]:
from pygradu import gridify
import shapely.geometry
import pandas as pd
import numpy as np
import importlib

%time grid = gridify.area_to_grid(side_length=5000)

Original NE: POINT (31.5 66.8)
Original SW: POINT (8.5 53)
Transformed NE: (697681.2863658136, 7416224.9393330095)
Transformed SW: (-735260.4063631196, 6033964.466017238)
col_count 286
row_count 276
grid_size 78936
CPU times: user 2.76 ms, sys: 2.98 ms, total: 5.74 ms
Wall time: 6.35 ms


In [2]:
# Import dataset

ais = pd.read_csv('data/datasets/training_set.csv', parse_dates = ['timestamp'])
ais = ais.sort_values(by=['mmsi', 'timestamp'])
ais.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,timestamp,mmsi,lat,lon,speed,course,heading,turnrate,breadth,vessel_type,vessel_max_speed,draft,power,dwt,iceclass
0,0,2017-11-02 11:19:07,205366000,54.347242,9.991140,4.01,56.6,56.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
1,1,2017-11-02 11:30:58,205366000,54.360348,10.025337,4.17,77.5,81.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
2,2,2017-11-02 11:37:58,205366000,54.359573,10.049203,4.48,92.4,92.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
3,3,2017-11-02 11:44:07,205366000,54.365497,10.071320,3.81,44.9,46.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA
4,4,2017-11-02 11:56:08,205366000,54.370437,10.109498,2.73,103.6,103.0,0.0,21.33,T,16.0,8.18,5820.0,13289.0,IA


In [3]:
ais = ais[ais.speed > 0.1]

In [4]:
# Remove unnecessary columns
%time ais_subset = ais[['timestamp', 'mmsi', 'lat', 'lon', 'speed', 'vessel_type']]

CPU times: user 185 ms, sys: 72.5 ms, total: 258 ms
Wall time: 258 ms


In [5]:
%time ais_subset['grid_point'] = ais_subset.apply(lambda row: grid.get_grid_point(row.lat, row.lon), axis=1)

CPU times: user 5min 32s, sys: 5.81 s, total: 5min 38s
Wall time: 5min 42s


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
%time ais_subset['node'] = ais_subset.apply(lambda row: grid.get_grid_position(row), axis=1)

CPU times: user 6min 56s, sys: 26.5 s, total: 7min 22s
Wall time: 7min 52s


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
importlib.reload(gridify)

%time avg_speeds = gridify.get_avg_speeds(ais_subset)
avg_speeds.head()

CPU times: user 51.6 s, sys: 1.98 s, total: 53.6 s
Wall time: 1min 1s


,-200,-197,-196,-195,-194,-193,76,77,78,79,...,68947,68948,69212,69213,69219,69487,69488,69495,69763,71160
CONT,NaN,NaN,6.281563,6.134489,5.101567,5.633553,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.680829,NaN,NaN,NaN,NaN,NaN
GC,1.394444,4.195000,5.298432,5.314470,4.799957,5.213959,3.704,4.106,4.1125,4.263636,...,1.199451,3.545102,5.569362,5.540676,2.613311,3.509783,4.548291,0.437899,1.026824,22.33
PAS,NaN,4.833333,4.728333,4.965000,4.224000,4.320000,NaN,NaN,4.6300,4.722222,...,1.780000,3.885000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RORO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
T,NaN,NaN,5.837040,5.819038,4.903045,5.430000,NaN,NaN,NaN,NaN,...,NaN,NaN,5.717500,5.884000,2.265556,2.830000,4.531250,NaN,1.230000,NaN


In [8]:
avg_speeds.to_dict()[354]

{'CONT': nan,
 'GC': 4.114166666666667,
 'PAS': 4.644285714285715,
 'RORO': nan,
 'T': nan}

In [ ]:
#avg_speeds.to_csv('speed_model_5km.csv')